In [1]:
from keras.applications.mobilenet import preprocess_input
from keras.models import load_model
from keras.preprocessing.image import img_to_array, array_to_img
from keras.applications.mobilenet import decode_predictions, preprocess_input
from PIL import Image
import numpy as np
from imagehash import phash


IMAGE_DIMS = (224, 224)
TREE_FROG_IDX = 31
TREE_FROG_STR = "tree_frog"


# I'm pretty sure I borrowed this function from somewhere, but cannot remember
# the source to cite them properly.
def hash_hamming_distance(h1, h2):
    s1 = str(h1)
    s2 = str(h2)
    
    print(s1)
    print(s2)
    
    return sum(map(lambda x: 0 if x[0] == x[1] else 1, zip(s1, s2)))


def is_similar_img(path1, path2):
    image1 = Image.open(path1)
    image2 = Image.open(path2)

    dist = hash_hamming_distance(phash(image1), phash(image2))
    return dist <= 2


def prepare_image(image, target=IMAGE_DIMS):
    # if the image mode is not RGB, convert it
    if image.mode != "RGB":
        image = image.convert("RGB")

    # resize the input image and preprocess it
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    # return the processed image
    return image


def create_img(img_path, img_res_path, model_path, target_str, target_idx, des_conf=0.95):
    test = Image.open(img_path).resize(IMAGE_DIMS)
    test = prepare_image(test)

    print(test.shape)
    # TODO: YOUR SOLUTION HERE


    test = test.reshape((224,224,3))
    img = array_to_img(test)
    img.save(img_res_path)
    
def get_predictions(image):
    preds = model.predict(image)
    dec_preds = decode_predictions(preds)[0]
    _, label1, conf1 = decode_predictions(preds)[0][0]
    return label1, conf1, dec_preds

model = load_model("./model.h5")
create_img("./img/trixi.png", "./trixi_frog.png", "./model.h5", TREE_FROG_STR, TREE_FROG_IDX)
assert is_similar_img("./img/trixi.png", "./trixi_frog.png")

frog_mat = prepare_image(Image.open("./img/trixi.png").resize(IMAGE_DIMS))
            
# read the image in PIL format
frog_label, frog_conf, top_preds = get_predictions(frog_mat)

print(frog_label)

Using TensorFlow backend.
/home/robert/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(1, 224, 224, 3)
963769983334ddc2
963769983334ddc2
malinois


In [2]:
decode_predictions(model.predict(frog_mat))

[[('n02105162', 'malinois', 0.51700354),
  ('n02106662', 'German_shepherd', 0.48188373),
  ('n02105412', 'kelpie', 0.0005297822),
  ('n02091467', 'Norwegian_elkhound', 0.00027945824),
  ('n02109047', 'Great_Dane', 7.9423575e-05)]]

In [22]:
from keras import backend as K

model_input_layer = model.layers[0].input
model_output_layer = model.layers[-1].output

object_type_to_fake = 31

# Load the image to hack

test = Image.open("./img/trixi.png").resize(IMAGE_DIMS)
test = prepare_image(test)

# Add a 4th dimension for batch size (as Keras expects)
original_image = test

# Pre-calculate the maximum change we will allow to the image
# We'll make sure our hacked image never goes past this so it doesn't look funny.
# A larger number produces an image faster but risks more distortion.
max_change_above = original_image + 0.1
max_change_below = original_image - 0.1


hacked_image = np.copy(original_image)

In [23]:


# Create a copy of the input image to hack on

# How much to update the hacked image in each iteration
learning_rate = 1e7

# Define the cost function.
# Our 'cost' will be the likelihood out image is the target class according to the pre-trained model
cost_function = model_output_layer[0, object_type_to_fake]

# We'll ask Keras to calculate the gradient based on the input image and the currently predicted class
# In this case, referring to "model_input_layer" will give us back image we are hacking.
gradient_function = K.gradients(cost_function, model_input_layer)[0]

# Create a Keras function that we can call to calculate the current cost and gradient
grab_cost_and_gradients_from_model = K.function([model_input_layer, K.learning_phase()], [cost_function, gradient_function])

cost = 0.0
iter = 0

# In a loop, keep adjusting the hacked image slightly so that it tricks the model more and more
# until it gets to at least 80% confidence
while True:
    # Check how close the image is to our target class and grab the gradients we
    # can use to push it one more step in that direction.
    # Note: It's really important to pass in '0' for the Keras learning mode here!
    # Keras layers behave differently in prediction vs. train modes!
    cost, gradients = grab_cost_and_gradients_from_model([hacked_image, 0])

    # Move the hacked image one step further towards fooling the model
    hacked_image += gradients * learning_rate

    # Ensure that the image doesn't ever change too much to either look funny or to become an invalid image
    hacked_image = np.clip(hacked_image, max_change_below, max_change_above)
    hacked_image = np.clip(hacked_image, -1, 1)

    iter += 1
    learning_rate *= 0.99
    if iter % 100 == 0:
        _, label1, conf1 = decode_predictions(model.predict(hacked_image))[0][0]
        print("{} {}".format(label1, conf1 * 100))
        print("{}".format(model.predict(hacked_image)[0][31]))
        if "tree_frog" in label1 and conf1 >= 0.95:
            break

shower_cap 25.31420886516571
0.1320762187242508
tree_frog 17.966926097869873
0.17966926097869873
sunglasses 20.077967643737793
0.0014713054988533258
Italian_greyhound 51.356059312820435
1.8276870150657487e-06
Italian_greyhound 82.85401463508606
0.003349896287545562
German_shepherd 41.97295904159546
7.492299482692033e-05
tree_frog 93.64550709724426
0.9364550709724426
bloodhound 46.38504385948181
1.994923150050454e-05
tree_frog 100.0
1.0


In [24]:
model.predict(hacked_image)[0][31]

1.0

In [25]:
img = hacked_image[0]
img = img.reshape((224,224,3))
img = array_to_img(img)
img.save("hacked-image3.png")
assert is_similar_img("./img/trixi.png", "./hacked-image3.png")

963769983334ddc2
963769983334ddc2
